*Created by Petteri Nevavuori <<petteri.nevavuori@gmail.com>>.*

---

# I. Building a General Purpose Blockchain

In this notebook we'll build a general purpose centralized blockhain as the first step into the realm of blockchains, distributed ledgers and smart contracts. We'll showcase the fundamental aspects of block-related functionalities. The implementation follows closely on the premises of course Blockchain A-Z found in Udemy but not identically. This is the first module of the course. We will utilize the Python-based Flask web-server package to serve the general purpose blockchain. The development itself will be separated to several stages.

While subsequent notebooks contain notes on the concepts, a suggested starter to read is an article by Kirill Eremenko called [How does Bitcoin / Blockchain Mining work?](https://medium.com/swlh/how-does-bitcoin-blockchain-mining-work-36db1c5cb55d).

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Building-the-Blockhain" data-toc-modified-id="1.-Building-the-Blockhain-1">1. Building the Blockhain</a></span></li><li><span><a href="#2.-Creating-the-Web-App" data-toc-modified-id="2.-Creating-the-Web-App-2">2. Creating the Web App</a></span><ul class="toc-item"><li><span><a href="#2.1-Testing-the-Blockchain-App" data-toc-modified-id="2.1-Testing-the-Blockchain-App-2.1">2.1 Testing the Blockchain App</a></span></li></ul></li><li><span><a href="#3.-Mining--Blocks" data-toc-modified-id="3.-Mining--Blocks-3">3. Mining  Blocks</a></span><ul class="toc-item"><li><span><a href="#3.1-Testing-the-Chain-Validity" data-toc-modified-id="3.1-Testing-the-Chain-Validity-3.1">3.1 Testing the Chain Validity</a></span></li></ul></li></ul></div>

## 1. Building the Blockhain

The first thing of course is to build the blockchain. This blockchain utilizes a [proof-of-work](https://en.wikipedia.org/wiki/Proof-of-work_system) (POW) to validate the blocks. Essentially the POW states a problem that requires work to solve it and a target level for acceptance of the proof. With Bitcoins this means iteratively brute-forcing through [nonces](https://en.wikipedia.org/wiki/Cryptographic_nonce) to find the [SHA-256 hash](https://en.wikipedia.org/wiki/SHA-2) with four leading zeros. The probability of selecting a proper hexadecimal string with of 64 characters randomly is:

In [1]:
print("{:.5f}%".format(int('0000'+'F'*60,16)/int('F'*64,16)*100))

0.00153%


Our implementation's `proof` of a block essentially equals the nonce. This means that to produce a valid proof a number of iterations have to be tried out.

In our blockchain the block-wise proofs are calculated with the proof values of the previous block. This means that *changing the a proof value mid-chain would require re-calculating i.e. re-mining the blocks* to have new proofs and thus re-calculating the block-wise hashes. Because our initial blockchain does not have any data associated with it just yet, linking the proofs and blocks separately serves as a safety measure to ensure the tampering of our distributed ledger would be nearly impossible.

In [1]:
import datetime
import hashlib
import json


class Blockchain:

    def __init__(self):
        
        self.chain = []
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        "Create a block."

        block = {
            'index': len(self.chain),
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
        }
        self.chain.append(block)
        
        return block

    def get_previous_block(self):
        
        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"
        
        return (hashlib
                .sha256(str(next_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True).encode()
        
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['proof']

            if self.hash_proof(previous_proof, next_proof)[:4] != '0000':
                
                return False

            return True

## 2. Creating the Web App

We will use Flask to create an accessible chain that is able to handle GET and POST requests for our chain accordingly. The endpoints for our web app are:

- ``[\blocks] POST``: Mine a single Block and save it to the centralized chain.
- ``[\blocks] GET``: Validate and retrieve the full chain of Blocks.

We will also use a distinct class to easen up the use of the server in a manner which allows for non-blocking use of the notebook. Thus we run the server in a separate thread. We will also wrap the request calls as more easily called functions of the app. The ``BlockchainApp`` is usable as a context manager to ensure that the distinct thread is closed.

An important point to notice is that the ``Blockchain`` instance will live separately from the ``BlockchainApp``. This means that while the ``BlockchainApp`` is re-initialized over and over, the actual Flask server accesses the ``Blockchain`` instance initialized at the same time with the server. Thus, the Blocks mined are persisted until the Python kernel is restarted.

In [2]:
import threading
import requests

from flask import Flask, request, jsonify
from werkzeug.serving import run_simple


class BlockchainApp:

    def __init__(self, host='localhost', port=5000, chain=Blockchain):

        self.host = host
        self.port = port
        self.chain = chain()

        self.host_url = f'http://{self.host}:{self.port}'
        
        self.app = Flask(__name__)
        self.add_api_endpoints()
        
        self.thread = threading.Thread(
            target=run_simple,
            kwargs={
                'hostname': self.host,
                'port': self.port,
                'application': self.app}
        )

    def __enter__(self):

        self.start()

        return self

    def __exit__(self, *args):

        self.stop()
        
    def add_api_endpoints(self):
        "Add API endpoints to the Flask WebApp."
        
        self.app.add_url_rule(
            rule='/blocks',
            view_func=self.api_blocks,
            methods=['GET', 'POST']
        )
        self.app.add_url_rule(
            rule='/blocks/validate',
            view_func=self.api_validate,
        )
        self.app.add_url_rule(
            rule='/shutdown',
            view_func=self.api_shutdown,
        )

    def api_blocks(self):
        "Either retrieve the node's current chain or post a new block to the chain."

        if request.method == 'POST':

            prev_block = self.chain.get_previous_block()
            prev_hash = self.chain.hash_block(prev_block)
            prev_proof = prev_block['proof']

            proof = self.chain.proof_of_work(prev_proof)

            block = self.chain.create_block(proof, prev_hash)

            response = {'message': 'Congratulations, you just mined a Block!',
                        'block': block}

            return (jsonify(response), 200)

        if request.method == 'GET':

            response = {'chain': self.chain.chain,
                        'length': len(self.chain.chain)}

            return (jsonify(response), 200)

    def api_validate(self):
        "Validate the chain"

        if self.chain.is_chain_valid(self.chain.chain):

            response = {'message': 'Chain is valid.',
                        'valid': True}
            return (jsonify(response), 200)

        else:

            response = {'message': 'Chain is not valid!',
                        'valid': False}
            return (jsonify(response), 500)

    def api_shutdown(self):
        "Shutdown the Flask WebApp"

        request.environ.get('werkzeug.server.shutdown')()

        return jsonify({'message': 'Shutting down'}), 200

    def start(self):
        "Start the Flask-based Blockchain WebApp."

        self.thread.start()

    def stop(self):
        "Stop the Flask-based Blockchain WebApp."

        if self.thread.is_alive():

            return requests.get(f'{self.host_url}/shutdown')

### 2.1 Testing the Blockchain App

Let's then give the app a test run!

In [4]:
with BlockchainApp() as blockchain_app:

    print(blockchain_app.__class__.__name__)

    response = requests.post(f'{blockchain_app.host_url}/blocks')
    print(json.dumps(response.json(), indent=2))

    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print(json.dumps(response.json(), indent=2))
    
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))

 * Running on http://localhost:5000/ (Press CTRL+C to quit)


BlockchainApp


127.0.0.1 - - [17/May/2018 11:23:01] "POST /blocks HTTP/1.1" 200 -


{
  "block": {
    "index": 1,
    "previous_hash": "aa037f09d677799a99bde71a5c39672432ec45b571c61bf19e300293d696d247",
    "proof": 533,
    "timestamp": "2018-05-17 11:23:01.276072"
  },
  "message": "Congratulations, you just mined a Block!"
}


127.0.0.1 - - [17/May/2018 11:23:02] "GET /blocks HTTP/1.1" 200 -


{
  "blockchain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "timestamp": "2018-05-17 11:23:00.257932"
    },
    {
      "index": 1,
      "previous_hash": "aa037f09d677799a99bde71a5c39672432ec45b571c61bf19e300293d696d247",
      "proof": 533,
      "timestamp": "2018-05-17 11:23:01.276072"
    }
  ],
  "length": 2
}


127.0.0.1 - - [17/May/2018 11:23:03] "GET /blocks/validate HTTP/1.1" 200 -


{
  "message": "Chain is valid.",
  "valid": true
}


127.0.0.1 - - [17/May/2018 11:23:04] "GET /shutdown HTTP/1.1" 200 -


## 3. Mining  Blocks

Then we'll mine several blocks. This means that we'll start the server and make a few requests to the `/blocks` with a POST request.

In [5]:
with BlockchainApp() as blockchain_app:

    for _ in range(4):

        response = requests.post(f'{blockchain_app.host_url}/blocks')
        print("Proof={}".format(response.json()['block']['proof']))

    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print("Chain={}".format(json.dumps(response.json(), indent=2)))
    
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2018 11:23:35] "POST /blocks HTTP/1.1" 200 -


Proof=533


127.0.0.1 - - [17/May/2018 11:23:36] "POST /blocks HTTP/1.1" 200 -


Proof=45293


127.0.0.1 - - [17/May/2018 11:23:37] "POST /blocks HTTP/1.1" 200 -


...

127.0.0.1 - - [17/May/2018 11:23:39] "GET /blocks HTTP/1.1" 200 -


Chain={
  "blockchain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "timestamp": "2018-05-17 11:23:34.489424"
    },
    {
      "index": 1,
      "previous_hash": "0898ebe729b4094d29e88a37666eea804283b358d7ef8f71b5f4e3569f71c23c",
      "proof": 533,
      "timestamp": "2018-05-17 11:23:35.499157"
    },
    {
      "index": 2,
      "previous_hash": "03aa5640ade8534dc1419b29413d8908c822c090079080139415e37c3aa89903",
      "proof": 45293,
      "timestamp": "2018-05-17 11:23:36.636971"
    },
    {
      "index": 3,
      "previous_hash": "6683bd4452b6f2acb9719d7cddb684bf8a5cc1ba2b56f2ab356db9731e1f792c",
      "proof": 21391,
      "timestamp": "2018-05-17 11:23:37.722398"
    },
    {
      "index": 4,
      "previous_hash": "7658422de8b132c643537bd9afd585c535aeb2f8b655264d5a0ab91971fa2747",
      "proof": 8018,
      "timestamp": "2018-05-17 11:23:38.760560"
    }
  ],
  "length": 5
}


127.0.0.1 - - [17/May/2018 11:23:40] "GET /blocks/validate HTTP/1.1" 200 -


{
  "message": "Chain is valid.",
  "valid": true
}


127.0.0.1 - - [17/May/2018 11:23:41] "GET /shutdown HTTP/1.1" 200 -


If you compare the first two Blocks with the test run you can see that the hashes and proofs match. This is intended, as already stated, as the ``Blockchain`` lives centralized in the server for this showcase and the ``BlockchainApp`` only makes calls to that server.

### 3.1 Testing the Chain Validity

We also want to see if changing the proof mid-chain breaks the chain. For this we must access the initialized ``Blockchain`` directly as the miniserver doesn't have a method for altering the chain - which is a wise starting point!

In [6]:
with BlockchainApp() as blockchain_app:
    
    for _ in range(4):

        response = requests.post(f'{blockchain_app.host_url}/blocks')
        print("Proof={}".format(response.json()['block']['proof']))

    original = blockchain_app.chain.chain[1]['proof']
    blockchain_app.chain.chain[1]['proof'] = 12345
    print('Proofs: Original={}, Forged={}'.format(original,blockchain_app.chain.chain[1]['proof']))
    
    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print(json.dumps(response.json(), indent=2))
    
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))
    
    proof_1 = blockchain_app.chain.chain[1]['proof']
    proof_2 = blockchain_app.chain.chain[2]['proof']

    forged_hash = blockchain_app.chain.hash_proof(proof_1, proof_2)

    print("Hashes: \n\tOriginal =\t{}\n\tForged =\t{}".format(blockchain_app.chain.chain[2]['previous_hash'],
                                                              forged_hash))


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2018 11:24:12] "POST /blocks HTTP/1.1" 200 -


Proof=533


127.0.0.1 - - [17/May/2018 11:24:13] "POST /blocks HTTP/1.1" 200 -


Proof=45293


127.0.0.1 - - [17/May/2018 11:24:14] "POST /blocks HTTP/1.1" 200 -


...

127.0.0.1 - - [17/May/2018 11:24:16] "GET /blocks HTTP/1.1" 200 -


{
  "blockchain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "timestamp": "2018-05-17 11:24:11.501229"
    },
    {
      "index": 1,
      "previous_hash": "3709200a7104364db3f4c7f0464e2b478ceff4a6ec9ef8c494ca946891fdb796",
      "proof": 12345,
      "timestamp": "2018-05-17 11:24:12.509229"
    },
    {
      "index": 2,
      "previous_hash": "aae67119ebb026be70cf1638c0b0e9aa3aa316029e90c05273e37254cc6ac9f8",
      "proof": 45293,
      "timestamp": "2018-05-17 11:24:13.646387"
    },
    {
      "index": 3,
      "previous_hash": "dcce5c492011de9724ada1bbd8a59f0679cca429d199558930efacc158207772",
      "proof": 21391,
      "timestamp": "2018-05-17 11:24:14.739003"
    },
    {
      "index": 4,
      "previous_hash": "4c9dd27b0001c1a16619ace602b9a08073e42ccd5482e562fb10b6ef163dfe5a",
      "proof": 8018,
      "timestamp": "2018-05-17 11:24:15.789439"
    }
  ],
  "length": 5
}


127.0.0.1 - - [17/May/2018 11:24:17] "GET /blocks/validate HTTP/1.1" 500 -


{
  "message": "Chain is not valid!",
  "valid": false
}
Hashes: 
	Original =	aae67119ebb026be70cf1638c0b0e9aa3aa316029e90c05273e37254cc6ac9f8
	Forged =	b3e1a78ae4b5cd947a6690f4a5ee9b3a71a13a2df00d77cbc4c904c9b2ac2deb


127.0.0.1 - - [17/May/2018 11:24:18] "GET /shutdown HTTP/1.1" 200 -


And so we indeed have invalidated the chain! The avalanche effect of the hash-function is clearly visible in the last few lines of the above printout. The hashes with the original proof and the forged one differ substantially from each other. This is what makes even slight forgeries impossible without comporising the validity of the chain.